# Asthma Hospitalisations | Processing

## Set Up

Ensure that the required libraries are available by running the below code in the terminal before execution:
- pip install pandas


Execute the following in the jupyter notebook before execution to ensure that the required libraries are imported:

In [4]:
import pandas as pd

## Load Dataset

In [5]:
df = pd.read_csv('raw.csv')

## Data Manipulation

In [6]:
# Rename columns to match Air Quality data set.
df = df.rename(columns={'LHD': 'lhd', 'Period': 'financial year', 'Sex': 'sex'})                                    # Rename columns
df.columns = df.columns.str.lower()                                                                                 # Lowercase column names

# Remove ' LHD' for Local Health District values.
df['lhd'] = df['lhd'].str.replace(' LHD', '')                                                                       # Remove ' LHD' from lhd column

# Remove rows representing state-wide aggregated data.
df = df.dropna(subset=['lhd'])                                                                                      # Remove rows with missing values in 'lhd' column
df = df[~df['lhd'].str.contains('All')]                                                                             # Remove rows with 'All' in 'lhd' column

# Remove columns holding Confidence Interval data.
df = df.loc[:, ~df.columns.str.contains('% ci')]                                                                    # Remove columns with '% ci' in the name

# Reformat 'financial year' values from XX/YY to XXXX/YYYY.
df['financial year'] = df['financial year'].apply(                                                                  # Add '20' to the start of each year value
    lambda x: f'20{x[:2]}/20{x[3:]}' if isinstance(x, str) else x
)

# Remove 'risk group' column.
df = df.drop(columns=['risk group'])                                                                                 # Remove 'risk group' column

# View the cleaned data.
df.head()                                                                                                           # Display the first few rows of the cleaned data

,sex,lhd,financial year,"rate per 100,000 population"
0,Males,Sydney,2001/2002,123.6
1,Males,Sydney,2002/2003,112.8
2,Males,Sydney,2003/2004,120.0
3,Males,Sydney,2004/2005,124.6
4,Males,Sydney,2005/2006,117.6


In [7]:
# Pivot the dataframe to have 'sex' as columns
df = df.pivot_table(index=['financial year', 'lhd'], columns='sex', values='rate per 100,000 population').reset_index()

# Rename the columns to match the desired format
df.columns.name = None
df = df.rename(columns={
    'Persons': 'Persons rate per 100,000 population',
    'Males': 'Male rate per 100,000 population', 
    'Females': 'Female rate per 100,000 population'
})

# View the DataFrame
df.head() 

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
0,2001/2002,Central Coast,173.850,125.825,152.125000
1,2001/2002,Far West,325.500,245.375,298.275000
2,2001/2002,Hunter New England,158.725,122.150,130.033333
3,2001/2002,Illawarra Shoalhaven,84.425,76.475,81.550000
4,2001/2002,Mid North Coast,150.600,136.125,144.450000


## Set Date Range

Set the range of financial years from 2014/2015 to 2023/2024.

In [9]:
# Add rows for each LHD for the missing years until 2023/2024.
missing_rows = []                                                                                                   # Create a list to store the missing rows.
lhds = df['lhd'].unique()                                                                                           # Get the unique values in the 'lhd' column.
years = [f"{year}/{year + 1}" for year in range(2014, 2024)]                                                        # Create a list of years from 2014/2015 to 2023/2024.

for lhd in lhds:
    for year in years:
        if not ((df['lhd'] == lhd) & (df['financial year'] == year)).any():                                         # Check if the row is missing.
            missing_rows.append({                                                                                   # Append the missing row to the list.
                'lhd': lhd, 
                'financial year': year, 
                'Female rate per 100,000 population': None,
                'Male rate per 100,000 population': None,
                'Persons rate per 100,000 population': None,
            })          

# Create a DataFrame from the missing rows and concatenate it to the original DataFrame
if missing_rows:
    df_missing = pd.DataFrame(missing_rows)                                                                         # Create a DataFrame from the missing rows.
    df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.

# Sort the DataFrame by 'lhd' and 'financial year'.
df = df.sort_values(by=['lhd', 'financial year']).reset_index(drop=True)                                           # Sort the DataFrame by 'lhd' and 'financial year'.

# View the DataFrame.
df.tail()                                                                                                          # View the last 5 rows of the DataFrame.

C:\Users\matth\AppData\Local\Temp\ipykernel_8952\186155949.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.


,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
364,2019/2020,Western Sydney,173.450,99.850,138.325
365,2020/2021,Western Sydney,128.075,117.075,123.325
366,2021/2022,Western Sydney,103.775,91.500,98.450
367,2022/2023,Western Sydney,NaN,NaN,NaN
368,2023/2024,Western Sydney,NaN,NaN,NaN


Fill missing values using linear interpolation.

In [10]:
df['Female rate per 100,000 population'] = df['Female rate per 100,000 population'].interpolate()                                       # Fill missing values using linear interpolation.
df['Male rate per 100,000 population'] = df['Male rate per 100,000 population'].interpolate()                                        # Fill missing values using linear interpolation.
df['Persons rate per 100,000 population'] = df['Persons rate per 100,000 population'].interpolate()                                     # Fill missing values using linear interpolation.

## Output Processed Dataset

In [11]:
# Save the processed data to a new CSV file.
df.to_csv('processed.csv', index=False)                                                                            # Save the processed data to a new CSV file.

# View the DataFrame.
df.head()                                                                                                          # View the first 5 rows of the DataFrame.

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
0,2001/2002,Central Coast,173.850,125.825,152.125
1,2002/2003,Central Coast,149.150,111.575,131.550
2,2003/2004,Central Coast,143.775,96.925,122.950
3,2004/2005,Central Coast,120.325,121.025,121.750
4,2005/2006,Central Coast,126.650,109.050,119.550
